In [2]:
import pickle
import sys
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as LR 
from sklearn.ensemble import HistGradientBoostingRegressor as HGBR
from sklearn.metrics import mean_absolute_error

sys.path.append('../src/')
import importlib
import SpacedRepition
importlib.reload(SpacedRepition)
from SpacedRepition import SpacedRepetitionModel as SRM


In [3]:
with open('../cleaned/1015_train_test.pkl', 'rb') as handle:
    train_test = pickle.load(handle)

In [4]:
test = train_test['test']
train = train_test['train']
del train_test

In [5]:
test.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
11568803,1.0,1363009539,2829619,u:dHVF,fr,en,ccf02ea2e390aea793ebf62b0c490075,<*sf>/indispensable<adj><mf><*numb>,2,2,...,0.9999,32.750220,-0.000144,274.0,0,1,1.732051,1.000000,1,0.666667
11568804,1.0,1363009539,1974004,u:dHVF,fr,en,97835f097be96da3ccb1e9b014b482fc,<*sf>/faux<adj><*gndr><*numb>,3,3,...,0.9999,22.847269,-0.000144,274.0,0,1,2.000000,1.000000,1,0.714286
11568805,1.0,1363009539,1974004,u:dHVF,fr,en,9c5f150b7ade77c73c8042af28b65ac0,<*sf>/tel<adj><*gndr><*numb>,2,1,...,0.9999,22.847269,-0.000144,274.0,1,1,1.414214,1.414214,1,0.500000
11568806,1.0,1363009539,1974004,u:dHVF,fr,en,1932e8abc8d82e23f343cdc4d84b8bbe,<*sf>/juste<adj><mf><*numb>,1,1,...,0.9999,22.847269,-0.000144,274.0,0,1,1.414214,1.000000,1,0.600000
11568807,1.0,1363009539,2156799,u:dHVF,fr,en,1f143f9a64cb3dd127ffc89c71840f55,<*sf>/fin<adj><*gndr><*numb>,1,1,...,0.9999,24.962951,-0.000144,274.0,0,1,1.414214,1.000000,1,0.600000


In [6]:
train.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,...,0.9999,320.018924,-0.000144,274.0,2,2,2.236068,1.732051,1,0.600000
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,...,0.5000,320.018924,-1.000000,274.0,0,1,2.236068,1.000000,1,0.750000
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,...,0.9999,320.018924,-0.000144,274.0,1,1,2.236068,1.414214,1,0.666667
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,...,0.5000,320.018924,-1.000000,274.0,1,1,2.449490,1.414214,1,0.700000
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,...,0.9999,320.018924,-0.000144,274.0,0,1,2.236068,1.000000,1,0.750000


In [7]:
print(f"train shape is {train.shape}")
print(f"test shape is {test.shape}")

train shape is (11568803, 22)
test shape is (1285423, 22)


data mapping
t: delta_days
h: half_life
right aka inst.fv[0][1]: right_sqrt
wront: wrong_sqrt


In [8]:
srm = SRM(train, test,)

In [9]:
srm.train_set.columns

Index(['p_recall', 'timestamp', 'delta', 'user_id', 'learning_language',
       'ui_language', 'lexeme_id', 'lexeme_string', 'history_seen',
       'history_correct', 'session_seen', 'session_correct', 'p_recall_clip',
       'delta_days', 'log_p_recall_clip', 'half_life', 'wrong',
       'session_wrong', 'right_sqrt', 'wrong_sqrt', 'bias', 'a'],
      dtype='object')

In [10]:
srm.train_set.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,p_recall_clip,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,...,0.9999,320.018924,-0.000144,274.0,2,2,2.236068,1.732051,1,0.600000
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,...,0.5000,320.018924,-1.000000,274.0,0,1,2.236068,1.000000,1,0.750000
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,...,0.9999,320.018924,-0.000144,274.0,1,1,2.236068,1.414214,1,0.666667
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,...,0.5000,320.018924,-1.000000,274.0,1,1,2.449490,1.414214,1,0.700000
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,...,0.9999,320.018924,-0.000144,274.0,0,1,2.236068,1.000000,1,0.750000


In [11]:
p_lr = LR()

In [12]:
train_feats_prob= srm.train_set.delta.to_numpy().reshape((-1,1))
train_y_prob = srm.train_set.p_recall_clip.to_numpy().reshape((-1,1))

test_feats_prob= srm.test_set.delta.to_numpy().reshape((-1,1))
test_y_prob = srm.test_set.p_recall_clip.to_numpy().reshape((-1,1))


In [13]:
p_lr.fit(train_feats_prob, train_y_prob)
predict_y_prob = p_lr.predict(test_feats_prob)

In [14]:
baseline = srm.train_set.p_recall_clip.mean()
baseline_preds = (np.repeat(baseline, len(predict_y_prob))).reshape((len(test_y_prob), -1))

In [15]:
print(f"Baseline Pred({round(baseline,3)}):{mean_absolute_error(y_true= test_y_prob, y_pred=baseline_preds)}")
print(f"MAE Linear Reg: {mean_absolute_error(y_true= test_y_prob, y_pred= predict_y_prob)}")

Baseline Pred(0.896):0.17426775310262124
MAE Linear Reg: 0.17438588242066413


Half life time

In [16]:
train_feats_hl= srm.train_set.delta.to_numpy().reshape((-1,1))
train_y_hl = srm.train_set.half_life.to_numpy().reshape((-1,1))

test_feats_hl= srm.test_set.delta.to_numpy().reshape((-1,1))
test_y_hl = srm.test_set.half_life.to_numpy().reshape((-1,1))

In [17]:
hl_lr = LR()
hl_lr.fit(train_feats_hl, train_y_hl)
predict_y_hl = hl_lr.predict(test_feats_hl)

In [18]:
ts_delta = srm.test_set.delta_days.to_numpy()
ts_delta = ts_delta.reshape(len(ts_delta), -1)

In [19]:
predict_y_hl_prob = 2**(-ts_delta/ predict_y_hl)

In [20]:
print(f"Baseline Pred({round(baseline,3)}):{mean_absolute_error(y_true= test_y_prob, y_pred=baseline_preds)}")
print(f"MAE Linear Reg: {mean_absolute_error(y_true= test_y_prob, y_pred= predict_y_prob)}")
print(f"MAE Half-Life Approx:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_hl_prob)}")

Baseline Pred(0.896):0.17426775310262124
MAE Linear Reg: 0.17438588242066413
MAE Half-Life Approx:0.12252910550020452


In [21]:
hl_gb = HGBR(loss='absolute_error')
hl_gb.fit(train_feats_hl, train_y_hl)
predict_y_gb_hl = hl_gb.predict(test_feats_hl)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
predict_y_gb_hl = predict_y_gb_hl.reshape((len(predict_y_gb_hl), -1))
predict_y_gb_hl_prob = 2**(-ts_delta/ predict_y_gb_hl)

In [24]:
print(f"Baseline Pred({round(baseline,3)}):{mean_absolute_error(y_true= test_y_prob, y_pred=baseline_preds)}")
print(f"MAE Linear Reg: {mean_absolute_error(y_true= test_y_prob, y_pred= predict_y_prob)}")
print(f"MAE Half-Life Approx:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_hl_prob)}")
print(f"MAE GB Half-Life Approx:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_gb_hl_prob)}")

Baseline Pred(0.896):0.17426775310262124
MAE Linear Reg: 0.17438588242066413
MAE Half-Life Approx:0.12252910550020452
MAE GB Half-Life Approx:0.11832987127563882


In [31]:
srm.train_set['hist_per_correct'] = srm.train_set.history_correct / srm.train_set.history_seen
srm.test_set['hist_per_correct'] = srm.test_set.history_correct / srm.test_set.history_seen

In [32]:
srm.train_set.head()

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,...,delta_days,log_p_recall_clip,half_life,wrong,session_wrong,right_sqrt,wrong_sqrt,bias,a,hist_per_correct
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,...,320.018924,-0.000144,274.0,2,2,2.236068,1.732051,1,0.600000,0.666667
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,...,320.018924,-1.000000,274.0,0,1,2.236068,1.000000,1,0.750000,1.000000
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,...,320.018924,-0.000144,274.0,1,1,2.236068,1.414214,1,0.666667,0.800000
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,...,320.018924,-1.000000,274.0,1,1,2.449490,1.414214,1,0.700000,0.833333
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,...,320.018924,-0.000144,274.0,0,1,2.236068,1.000000,1,0.750000,1.000000


In [33]:
train_feats_regsv1 = ['delta', 'history_seen', 'history_correct']

train_feats_hl_histv1= srm.train_set[train_feats_regsv1].to_numpy().reshape((-1,len(train_feats_regsv1)))
train_y_hl_histv1 = srm.train_set.half_life.to_numpy().reshape((-1,1))

test_feats_hl_histv1= srm.test_set[train_feats_regsv1].to_numpy().reshape((-1,len(train_feats_regsv1)))
test_y_hl_histv1 = srm.test_set.half_life.to_numpy().reshape((-1,1))

In [34]:
train_feats_regsv2 = ['delta', 'hist_per_correct', 'history_seen']

train_feats_hl_histv2= srm.train_set[train_feats_regsv2].to_numpy().reshape((-1,len(train_feats_regsv2)))
train_y_hl_histv2 = srm.train_set.half_life.to_numpy().reshape((-1,1))

test_feats_hl_histv2= srm.test_set[train_feats_regsv2].to_numpy().reshape((-1,len(train_feats_regsv2)))
test_y_hl_histv2 = srm.test_set.half_life.to_numpy().reshape((-1,1))

In [35]:
hl_lr_histv1 = LR()
hl_lr_histv1.fit(train_feats_hl_histv1, train_y_hl_histv1)

hl_lr_histv2 = LR()
hl_lr_histv2.fit(train_feats_hl_histv2, train_y_hl_histv2)

LinearRegression()

In [37]:
predict_y_hl_histv1 = hl_lr_histv1.predict(test_feats_hl_histv1)
predict_y_hl_histv1 = predict_y_hl_histv1.reshape((len(predict_y_hl_histv1), -1))
predict_y_hl_histv1_prob = 2**(-ts_delta/ predict_y_hl_histv1)

predict_y_hl_histv2 = hl_lr_histv2.predict(test_feats_hl_histv2)
predict_y_hl_histv2 = predict_y_hl_histv2.reshape((len(predict_y_hl_histv2), -1))
predict_y_hl_histv2_prob = 2**(-ts_delta/ predict_y_hl_histv2)


In [38]:
print(f"Baseline Pred({round(baseline,3)}):{mean_absolute_error(y_true= test_y_prob, y_pred=baseline_preds)}")
print(f"MAE Linear Reg: {mean_absolute_error(y_true= test_y_prob, y_pred= predict_y_prob)}")
print(f"MAE GB Half-Life Approx:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_gb_hl_prob)}")
print(f"MAE Half-Life Approx:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_hl_prob)}")
print(f"MAE Half-Life Approx v1:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_hl_histv1_prob)}")
print(f"MAE Half-Life Approx v2:{mean_absolute_error(y_true= test_y_prob, y_pred=predict_y_hl_histv2_prob)}")

Baseline Pred(0.896):0.17426775310262124
MAE Linear Reg: 0.17438588242066413
MAE GB Half-Life Approx:0.11832987127563882
MAE Half-Life Approx:0.12252910550020452
MAE Half-Life Approx v1:0.12250704547920378
MAE Half-Life Approx v2:0.12244516438374929


Seems like our best model is just the vanilla half life model which is cool i guess

We should probs make another notebook to handle some of the choosing logic that entails actually picking a set of cards

TODO Try different models, develop some output handler to define next set of cards chosen